
# Ejercicio: uso de un modelo entrenado en datos nuevos
En la Unidad 3, creamos un modelo básico que nos permitió encontrar la relación entre el tamaño del arnés de un perro y el tamaño de su bota. Mostramos cómo este modelo podría usarse para hacer una predicción sobre un perro nuevo, nunca antes visto.

Es común construir, entrenar y luego usar un modelo mientras estamos aprendiendo sobre el aprendizaje automático; pero en el mundo real, no queremos entrenar el modelo cada vez que queremos hacer una predicción.

Considere nuestro escenario de tienda de equipos para perros de avalanchas:

Queremos entrenar el modelo solo una vez, luego cargar ese modelo en el servidor que ejecuta nuestra tienda en línea.
Aunque el modelo está entrenado en un conjunto de datos que descargamos de Internet, en realidad queremos usarlo para estimar el tamaño de las botas de los perros de nuestros clientes que no están en este conjunto de datos.
¿Cómo podemos hacer esto?

Aquí, haremos lo siguiente:

# Crear un modelo básico
Guárdelo en el disco
Cargarlo desde el disco
Úselo para hacer predicciones sobre un perro que no estaba en el conjunto de datos de entrenamiento
Cargue el conjunto de datos
Comencemos abriendo el conjunto de datos desde el archivo.

In [1]:
import pandas
!pip install statsmodels
!curl -o graphing.py https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!curl -o doggy-boot-harness.csv https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/doggy-boot-harness.csv

# Load a file containing dog's boot and harness sizes
data = pandas.read_csv('doggy-boot-harness.csv')

# Print the first few rows
data.head()

You should consider upgrading via the 'C:\Users\kvlov\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 21511  100 21511    0     0  50444      0 --:--:-- --:--:-- --:--:-- 50495
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   838  100   838    0     0   2098      0 --:--:-- --:--:-- --:--:--  2105


,boot_size,harness_size,sex,age_years
0,39,58,male,12.0
1,38,58,male,9.6
2,37,52,female,8.6
3,39,58,male,10.2
4,38,57,male,7.8



# Crear y entrenar un modelo
Como hemos hecho antes, crearemos un modelo de regresión lineal simple y lo entrenaremos en nuestro conjunto de datos.

In [2]:
import statsmodels.formula.api as smf

# Fit a simple model that finds a linear relationship
# between boot size and harness size, which we can use later
# to predict a dog's boot size, given their harness size
model = smf.ols(formula = "boot_size ~ harness_size", data = data).fit()

print("Model trained!")

Model trained!



# Guardar y cargar un modelo
Nuestro modelo está listo para usar, pero aún no lo necesitamos. Guardémoslo en el disco.

In [3]:
import joblib

model_filename = './avalanche_dog_boot_model.pkl'
joblib.dump(model, model_filename)

print("Model saved!")

Model saved!



Cargar nuestro modelo es igual de fácil:

In [4]:

model_loaded = joblib.load(model_filename)

print("We have loaded a model with the following parameters:")
print(model_loaded.params)

We have loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64



# Ponlo junto
En nuestro sitio web, querremos tomar el arnés del perro de nuestro cliente y luego calcular el tamaño de la bota de su perro usando el modelo que ya hemos entrenado.

Juntemos todo aquí para crear una función que cargue el modelo desde el disco y luego lo use para predecir la altura del tamaño de la bota del perro de nuestro cliente.

In [5]:
# Let's write a function that loads and uses our model
def load_model_and_predict(harness_size):
    '''
    This function loads a pretrained model. It uses the model
    with the customer's dog's harness size to predict the size of
    boots that will fit that dog.

    harness_size: The dog harness size, in cm 
    '''

    # Load the model from file and print basic information about it
    loaded_model = joblib.load(model_filename)

    print("We've loaded a model with the following parameters:")
    print(loaded_model.params)

    # Prepare data for the model
    inputs = {"harness_size":[harness_size]} 

    # Use the model to make a prediction
    predicted_boot_size = loaded_model.predict(inputs)[0]

    return predicted_boot_size

# Practice using our model
predicted_boot_size = load_model_and_predict(45)

print("Predicted dog boot size:", predicted_boot_size)

We've loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64
Predicted dog boot size: 32.08575356590478



# Uso en el mundo real
Lo hicimos; podemos predecir el tamaño de la bota de un perro de avalanchas en función del tamaño de su arnés. Nuestro último paso es usar esto para advertir a las personas si podrían estar comprando botas para perros del tamaño incorrecto.

Como ejemplo, haremos una función que acepte el tamaño del arnés, el tamaño de las botas seleccionadas y devuelva un mensaje para el cliente. Integraríamos esta función en nuestra tienda online.

In [6]:
def check_size_of_boots(selected_harness_size, selected_boot_size):
    '''
    Calculates whether the customer has chosen a pair of doggy boots that 
    are a sensible size. This works by estimating the dog's actual boot 
    size from their harness size.

    This returns a message for the customer that should be shown before
    they complete their payment 

    selected_harness_size: The size of the harness the customer wants to buy
    selected_boot_size: The size of the doggy boots the customer wants to buy
    '''

    # Estimate the customer's dog's boot size
    estimated_boot_size = load_model_and_predict(selected_harness_size)

    # Round to the nearest whole number because we don't sell partial sizes
    estimated_boot_size = int(round(estimated_boot_size))

    # Check if the boot size selected is appropriate
    if selected_boot_size == estimated_boot_size:
        # The selected boots are probably OK
        return f"Great choice! We think these boots will fit your avalanche dog well."

    if selected_boot_size < estimated_boot_size:
        # Selected boots might be too small 
        return "The boots you have selected might be TOO SMALL for a dog as "\
               f"big as yours. We recommend a doggy boots size of {estimated_boot_size}."

    if selected_boot_size > estimated_boot_size:
        # Selected boots might be too big 
        return "The boots you have selected might be TOO BIG for a dog as "\
               f"small as yours. We recommend a doggy boots size of {estimated_boot_size}."
    

# Practice using our new warning system
check_size_of_boots(selected_harness_size=55, selected_boot_size=39)

We've loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64


'The boots you have selected might be TOO BIG for a dog as small as yours. We recommend a doggy boots size of 38.'


Cambie selected_harness_sizey selected_boot_sizeen el ejemplo anterior y vuelva a ejecutar la celda para ver esto en acción.

Resumen
¡Bien hecho! Hemos creado un sistema que puede predecir si los clientes están comprando botas para perros que no se ajustan a su perro de avalancha, basándose únicamente en el tamaño del arnés que están comprando.

En este ejercicio practicamos:

Crear modelos básicos
Entrenando, luego guardándolos en el disco
Cargándolos desde el disco
Hacer predicciones con ellos utilizando nuevos conjuntos de datos.